In [11]:
import requests
import re

In [16]:
# get html from imslp website
url = 'https://imslp.org/wiki/Ach_bleib_bei_uns,_Herr_Jesu_Christ,_BWV_649_(Bach,_Johann_Sebastian)'
def get_ids(url):
    response = requests.get(url)
    html = response.text

    # find all strings of the form "id='num-of-ratings-[0-9]+'" under the "Scores" section and extract the number
    html = html[html.find('id="Sheet_Music"'):]


    pdf_id = re.findall(r'id=\'num-of-ratings-[0-9]+\'', html)
    pdf_id = [int(re.findall(r'[0-9]+', r)[0]) for r in pdf_id]
    pdf_dict = {}
    for i in pdf_id:
        pdf_dict[i] = url
    return pdf_dict

print(get_ids(url))

{852606: 'https://imslp.org/wiki/Ach_bleib_bei_uns,_Herr_Jesu_Christ,_BWV_649_(Bach,_Johann_Sebastian)', 65816: 'https://imslp.org/wiki/Ach_bleib_bei_uns,_Herr_Jesu_Christ,_BWV_649_(Bach,_Johann_Sebastian)', 129112: 'https://imslp.org/wiki/Ach_bleib_bei_uns,_Herr_Jesu_Christ,_BWV_649_(Bach,_Johann_Sebastian)', 384538: 'https://imslp.org/wiki/Ach_bleib_bei_uns,_Herr_Jesu_Christ,_BWV_649_(Bach,_Johann_Sebastian)', 494815: 'https://imslp.org/wiki/Ach_bleib_bei_uns,_Herr_Jesu_Christ,_BWV_649_(Bach,_Johann_Sebastian)'}
